In [1]:
import numpy as np
import pandas as pd
import gdreg
import time
import matplotlib.pyplot as plt

# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
DATA_PATH='/n/groups/price/martin/WES_analysis/'
OUT_PATH = '/n/groups/price/martin/WES_analysis/toy_1K'

### Sample list (1K) and SNP list (10K) from WES_50K
- Plink file: /n/groups/price/UKBiobank/WES_50K/chr*_v1.SPB.hg19.bed
- Plink .frq file: /n/groups/price/UKBiobank/WES_50K/freq/chr*_v1.SPB.hg19.frq
- 340K unrelated inviduals: /n/groups/price/UKBiobank/sampleQC/samples_337K.txt
- Withdrawn samples: /n/groups/price/UKBiobank/download_500K/w14048_CURRENT.FID_IID.txt

In [3]:
# # Get 1K sample
# df_fam = pd.read_csv('/n/groups/price/UKBiobank/WES_50K/chr1_v1.SPB.hg19.fam', sep=' ', header=None)
# df_fam.columns = ['FID', 'IID', 'FATHER', 'MOTHER', 'SEX', 'PHEN']
# df_fam.index = ['%s_%s'%(x,y) for x,y in zip(df_fam['FID'], df_fam['IID'])]
# print('df_fam: %d'%len(df_fam.index))

# df_unrelated = pd.read_csv('/n/groups/price/UKBiobank/sampleQC/samples_337K.txt', sep=' ', header=None)
# df_unrelated.columns = ['FID', 'IID']
# df_unrelated.index = ['%s_%s'%(x,y) for x,y in zip(df_unrelated['FID'], df_unrelated['IID'])]
# print('df_unrelated: %d'%len(df_unrelated.index))

# df_remove = pd.read_csv(
#     '/n/groups/price/UKBiobank/download_500K/w14048_CURRENT.FID_IID.txt', sep=' ', header=None
# )
# df_remove.columns = ['FID', 'IID']
# df_remove.index = ['%s_%s'%(x,y) for x,y in zip(df_remove['FID'], df_remove['IID'])]
# print('df_remove: %d'%len(df_remove.index))

# # Subsample ID_list
# legit_id_list = sorted(set(df_fam.index) & set(df_unrelated.index) - set(df_remove.index))
# print('legit_id_list: %d'%len(legit_id_list))

# np.random.seed(0)
# id_1K_list = np.random.choice(legit_id_list, size=1000, replace=False)
# id_1K_list = sorted(set(id_1K_list))
# print('id_10K_list: %d'%len(id_1K_list))

# df_id_1K = df_fam.loc[id_1K_list, ['FID', 'IID']]
# df_id_1K.to_csv(OUT_PATH+'/ID1K_unrelated.txt', sep=' ', header=False, index=False)

In [4]:
# # Get 20K SNPs from first 10 CHRs (2K each, with some redundancy)
# BIM_FILE = '/n/groups/price/UKBiobank/WES_50K/chr@_v1.SPB.hg19.bim'
# FRQ_FILE = '/n/groups/price/UKBiobank/WES_50K/freq/chr@_v1.SPB.hg19.frq'
# snp_list = []
# for CHR in range(1, 11):
#     df_freq = pd.read_csv(FRQ_FILE.replace('@', '%d'%CHR), delim_whitespace=True)
#     df_freq = df_freq.loc[df_freq['MAF']>0.05]
#     snp_list.extend(df_freq['SNP'].values[:1500])
    
# with open(OUT_PATH+'/SNP15K_CHR1t10_MAFg5.txt', 'w') as f:
#     for snp in snp_list:
#         f.write('%s\n'%snp)

### Call get_toydata_genotype.sh to get .pgen files

In [5]:
# Check if the files are good
PGEN_FILE = OUT_PATH + '/chr@_v1.SPB.hg19.toy_1K'
dic_data = {}
for CHR in range(1,11):
    dic_data[CHR] = gdreg.util.read_pgen(PGEN_FILE.replace('@', '%s'%CHR))
    print(CHR, 'n_snp=%d'%dic_data[CHR]['pvar'].shape[0])
    if CHR==1:
        display(dic_data[CHR]['pvar'])
        display(dic_data[CHR]['psam'])
        display(dic_data[CHR]['afreq'])

1 n_snp=1400


,CHR,BP,SNP,REF,ALT
0,1,69761,1:69761:A:T,A,T
1,1,866511,1:931131:I:4.01,CCCCT,C
2,1,871334,1:935954:G:T,T,G
3,1,874816,1:939436:I:1,C,CT
4,1,876499,1:941119:A:G,G,A
...,...,...,...,...,...
1395,1,12939440,1:12879619:C:G,C,G
1396,1,12939467,1:12879646:C:T,C,T
1397,1,12939476,1:12879655:G:C,G,C
1398,1,12939546,1:12879725:I:1,C,CG


,FID,IID,SEX
0,4663783,4663783,2
1,3967789,3967789,1
2,1292546,1292546,2
3,1605909,1605909,1
4,1637792,1637792,2
...,...,...,...
995,1937047,1937047,2
996,1664399,1664399,1
997,3710395,3710395,1
998,4555321,4555321,1


,CHR,SNP,REF,ALT,MAF,OBS_CT
0,1,1:69761:A:T,A,T,0.087906,1968
1,1,1:931131:I:4.01,CCCCT,C,0.253638,1924
2,1,1:935954:G:T,T,G,0.275000,2000
3,1,1:939436:I:1,C,CT,0.056000,2000
4,1,1:941119:A:G,G,A,0.047547,1998
...,...,...,...,...,...,...
1395,1,1:12879619:C:G,C,G,0.123000,2000
1396,1,1:12879646:C:T,C,T,0.123500,2000
1397,1,1:12879655:G:C,G,C,0.117000,2000
1398,1,1:12879725:I:1,C,CG,0.114500,2000


2 n_snp=1322
3 n_snp=1326
4 n_snp=1341
5 n_snp=1311
6 n_snp=1341
7 n_snp=1375
8 n_snp=1320
9 n_snp=1307
10 n_snp=1280


### Create annotations

In [6]:
# # Create .annot.gz file
# df_annot = None
# for CHR in dic_data:
#     if df_annot is None:
#         df_annot = dic_data[CHR]['pvar']
#     else:
#         df_annot = pd.concat([df_annot, dic_data[CHR]['pvar']], axis=0)
        
# df_annot = df_annot[['CHR', 'SNP', 'BP']]
# df_annot['CM'] = 0
# df_annot['AN:ALL'] = 1
# df_annot['AN:CHR1'] = (df_annot['CHR']==1)*1
# df_annot['AN:ODD'] = ((df_annot['CHR'] % 2) == 1) * 1
# gdreg.util.write_annot(df_annot, OUT_PATH+'/toy.annot.gz')

In [7]:
# # Create .pannot.gz file
# df_annot = None
# for CHR in dic_data:
#     if df_annot is None:
#         df_annot = dic_data[CHR]['pvar']
#     else:
#         df_annot = pd.concat([df_annot, dic_data[CHR]['pvar']], axis=0)
        
# df_annot = df_annot[['CHR', 'SNP', 'BP']]
# df_annot['CM'] = 0
# df_annot['pAN:gene'] = 'non-pAN'
# # 5 genes for each CHR, each with 50 SNPs
# for CHR in dic_data:
#     for i in range(5):
#         snp_list = dic_data[CHR]['pvar']['SNP'][i*100:i*100+50]
#         df_annot.loc[df_annot['SNP'].isin(snp_list), 'pAN:gene'] = 'GENE_%d_%d'%(CHR,i)
# gdreg.util.write_annot(df_annot, OUT_PATH+'/toy.pannot.gz')

In [8]:
# # Create .pannot_hr.gz file
# df_annot = None
# for CHR in dic_data:
#     if df_annot is None:
#         df_annot = dic_data[CHR]['pvar']
#     else:
#         df_annot = pd.concat([df_annot, dic_data[CHR]['pvar']], axis=0)
        
# df_annot = df_annot[['CHR', 'SNP', 'BP']]
# df_annot['CM'] = 0

# # SNPs within 10-kb 
# iSNP_list = []
# jSNP_list = []
# v_SNP = df_annot['SNP'].values
# v_CHR = df_annot['CHR'].values
# v_BP = df_annot['BP'].values

# for i in range(v_SNP.shape[0]):
#     for j in range(i+1, v_SNP.shape[0]):
#         if ((v_BP[j] - v_BP[i])<1e4) & (v_CHR[i] == v_CHR[j]):
#             iSNP_list.append(i)
#             jSNP_list.append(j)
#         else:
#             break
        
# df_pannot = pd.DataFrame(data={
#     'CHR' : df_annot['CHR'].values[iSNP_list],
#     'SNP' : df_annot['SNP'].values[iSNP_list],
#     'BP' : df_annot['BP'].values[iSNP_list],
#     'CM' : df_annot['CM'].values[iSNP_list],
#     'pCHR' : df_annot['CHR'].values[jSNP_list],
#     'pSNP' : df_annot['SNP'].values[jSNP_list],
#     'pBP' : df_annot['BP'].values[jSNP_list],
#     'pCM' : df_annot['CM'].values[jSNP_list],
#     'pAN:proxy' : 1,
# })
            
# gdreg.util.write_annot(df_pannot, OUT_PATH+'/toy.pannot_hr.gz')

### Check if the .annot files are good

In [9]:
# Read .annot files
dic_annot = {}
dic_annot['annot'] = gdreg.util.read_annot(OUT_PATH+'/toy.annot.gz')
display(dic_annot['annot'])
dic_annot['pannot'] = gdreg.util.read_annot(OUT_PATH+'/toy.pannot.gz')
display(dic_annot['pannot'])
dic_annot['pannot_hr'] = gdreg.util.read_annot(OUT_PATH+'/toy.pannot_hr.gz')
display(dic_annot['pannot_hr'])

,CHR,SNP,BP,CM,AN:ALL,AN:CHR1,AN:ODD
0,1,1:69761:A:T,69761,0,1,1,1
1,1,1:931131:I:4.01,866511,0,1,1,1
2,1,1:935954:G:T,871334,0,1,1,1
3,1,1:939436:I:1,874816,0,1,1,1
4,1,1:941119:A:G,876499,0,1,1,1
...,...,...,...,...,...,...,...
13318,10,10:32922299:T:G,33211227,0,1,0,0
13319,10,10:32925874:A:G,33214802,0,1,0,0
13320,10,10:32928182:G:A,33217110,0,1,0,0
13321,10,10:32930080:T:G,33219008,0,1,0,0


,CHR,SNP,BP,pAN:gene
0,1,1:69761:A:T,69761,GENE_1_0
1,1,1:931131:I:4.01,866511,GENE_1_0
2,1,1:935954:G:T,871334,GENE_1_0
3,1,1:939436:I:1,874816,GENE_1_0
4,1,1:941119:A:G,876499,GENE_1_0
...,...,...,...,...
13318,10,10:32922299:T:G,33211227,non-pAN
13319,10,10:32925874:A:G,33214802,non-pAN
13320,10,10:32928182:G:A,33217110,non-pAN
13321,10,10:32930080:T:G,33219008,non-pAN


,CHR,SNP,BP,pCHR,pSNP,pBP,pAN:proxy
0,1,1:931131:I:4.01,866511,1,1:935954:G:T,871334,1
1,1,1:931131:I:4.01,866511,1,1:939436:I:1,874816,1
2,1,1:931131:I:4.01,866511,1,1:941119:A:G,876499,1
3,1,1:935954:G:T,871334,1,1:939436:I:1,874816,1
4,1,1:935954:G:T,871334,1,1:941119:A:G,876499,1
...,...,...,...,...,...,...,...
56977,10,10:32922299:T:G,33211227,10,10:32928182:G:A,33217110,1
56978,10,10:32922299:T:G,33211227,10,10:32930080:T:G,33219008,1
56979,10,10:32925874:A:G,33214802,10,10:32928182:G:A,33217110,1
56980,10,10:32925874:A:G,33214802,10,10:32930080:T:G,33219008,1


### Simulate .snpeff and .phen values

In [13]:
df_afreq = None
for CHR in dic_data:
    if df_afreq is None:
        df_afreq = dic_data[CHR]['afreq']
    else:
        df_afreq = pd.concat([df_afreq, dic_data[CHR]['afreq']], axis=0)

dic_coef = {
    'AN:ALL' : 1,
    'AN:CHR1' : 1,
    'AN:ODD' : 0.5,
    'pAN:gene|AN:CHR1' : 0.8,
    'pAN:proxy|AN:CHR1' : -0.2,
}

df_annot = dic_annot['annot'].copy()
df_annot['MAF'] = df_afreq['MAF'].values

AN_list = ['AN:ALL', 'AN:CHR1', 'AN:ODD']
pAN_list = ['pAN:gene', 'pAN:proxy']

In [21]:
# Sanity
n_rep = 10 
dic_summary = {}

for i in range(n_rep):
    print(i)
    df_effect = gdreg.simulate.simulate_snp_effect(
        df_annot, 
        dic_coef, 
        pannot_list = [dic_annot['pannot']], 
        pannot_hr_list = [dic_annot['pannot_hr']],
        p_causal = 1,
        alpha=-1,
        verbose=False,
        random_seed=i,
    )

    dic_summary[i] = gdreg.simulate.summarize_snp_effect(
        df_effect,
        df_annot, 
        pannot_list = [dic_annot['pannot']],
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False
    )
    
    df_effect.to_csv(OUT_PATH+'/sanity_rep%d.eff.gz'%i, sep='\t', index=False)
    dic_summary[i].to_csv(OUT_PATH+'/sanity_rep%d.eff_sum'%i, sep='\t', index=False)
    
# Summary 
temp_dic = dic_summary.copy()
for AN in AN_list:
    true_val = dic_coef[AN] if AN in dic_coef else 0
    v_tau = [temp_dic[x].loc[AN, 'tau'] for x in range(n_rep)]
    tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
    v_h2_ps = [temp_dic[x].loc[AN, 'h2_ps'] for x in range(n_rep)]
    h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s (true=%0.2g)'%(AN, true_val), 
            'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
            'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
        ))
    
for AN in AN_list:
    for pAN in pAN_list:
        true_val = dic_coef['%s|%s'%(pAN,AN)] if '%s|%s'%(pAN,AN) in dic_coef else 0
        v_rho = [temp_dic[x].loc[AN, 'rho|%s'%pAN] for x in range(n_rep)]
        rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
        v_r2_ps = [temp_dic[x].loc[AN, 'r2_ps|%s'%pAN] for x in range(n_rep)]
        r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
        print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
            'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
            'r2_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
        ))

0
1
2
3
4
5
6
7
8
9
       AN:ALL (true=1)        |   tau=2.7e-05 (SD=1.1e-06)   |  h2_ps=3.7e-05 (SD=1.7e-06)  
       AN:CHR1 (true=1)       |   tau=3.1e-05 (SD=4.3e-06)   |  h2_ps=7.1e-05 (SD=5.4e-06)  
      AN:ODD (true=0.5)       |   tau=1.4e-05 (SD=5.8e-07)   |  h2_ps=4.7e-05 (SD=2.3e-06)  
   pAN:gene|AN:ALL (true=0)   |    rho=0.0011 (SD=0.018)     |   r2_ps=0.0011 (SD=0.018)    
  pAN:proxy|AN:ALL (true=0)   |   rho=0.00036 (SD=0.0092)    |  r2_ps=0.00036 (SD=0.0092)   
 pAN:gene|AN:CHR1 (true=0.8)  |      rho=0.6 (SD=0.31)       |     r2_ps=0.6 (SD=0.31)      
pAN:proxy|AN:CHR1 (true=-0.2) |     rho=-0.08 (SD=0.038)     |    r2_ps=-0.08 (SD=0.038)    
   pAN:gene|AN:ODD (true=0)   |   rho=0.00068 (SD=0.00092)   |  r2_ps=0.00068 (SD=0.00092)  
  pAN:proxy|AN:ODD (true=0)   |   rho=-0.0057 (SD=0.0069)    |  r2_ps=-0.0057 (SD=0.0069)   


In [15]:
# Realistic
n_rep = 10 
dic_summary = {}

for i in range(n_rep):
    print(i)
    df_effect = gdreg.simulate.simulate_snp_effect(
        df_annot, 
        dic_coef, 
        pannot_list = [dic_annot['pannot']], 
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False,
        random_seed=i,
    )

    dic_summary[i] = gdreg.simulate.summarize_snp_effect(
        df_effect,
        df_annot, 
        pannot_list = [dic_annot['pannot']],
        pannot_hr_list = [dic_annot['pannot_hr']],
        verbose=False
    )
    
# Summary 
temp_dic = dic_summary.copy()
for AN in AN_list:
    true_val = dic_coef[AN] if AN in dic_coef else 0
    v_tau = [temp_dic[x].loc[AN, 'tau'] for x in range(n_rep)]
    tau_mean, tau_sd = np.mean(v_tau), np.std(v_tau)
    v_h2_ps = [temp_dic[x].loc[AN, 'h2_ps'] for x in range(n_rep)]
    h2_ps_mean, h2_ps_sd = np.mean(v_h2_ps), np.std(v_h2_ps)
    print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s (true=%0.2g)'%(AN, true_val), 
            'tau=%0.2g (SD=%0.2g)' % (tau_mean, tau_sd),
            'h2_ps=%0.2g (SD=%0.2g)' % (h2_ps_mean, h2_ps_sd),
        ))
    
for AN in AN_list:
    for pAN in pAN_list:
        true_val = dic_coef['%s|%s'%(pAN,AN)] if '%s|%s'%(pAN,AN) in dic_coef else 0
        v_rho = [temp_dic[x].loc[AN, 'rho|%s'%pAN] for x in range(n_rep)]
        rho_mean, rho_sd = np.mean(v_rho), np.std(v_rho)
        v_r2_ps = [temp_dic[x].loc[AN, 'r2_ps|%s'%pAN] for x in range(n_rep)]
        r2_ps_mean, r2_ps_sd = np.mean(v_rho), np.std(v_rho)
        print('{:^30s}|{:^30s}|{:^30s}'.format(
            '%s|%s (true=%0.2g)'%(pAN,AN,true_val), 
            'rho=%0.2g (SD=%0.2g)' % (rho_mean, rho_sd),
            'r2_ps=%0.2g (SD=%0.2g)' % (r2_ps_mean, r2_ps_sd),
        ))

0
1
2
3
4
       AN:ALL (true=1)        |   tau=2.8e-05 (SD=1.5e-06)   |  h2_ps=3.8e-05 (SD=2.2e-06)  
       AN:CHR1 (true=1)       |   tau=2.8e-05 (SD=7.8e-06)   |   h2_ps=7e-05 (SD=7.6e-06)   
      AN:ODD (true=0.5)       |   tau=1.3e-05 (SD=1.6e-06)   |  h2_ps=4.7e-05 (SD=2.9e-06)  
   pAN:gene|AN:ALL (true=0)   |    rho=0.0033 (SD=0.0057)    |   r2_ps=0.0033 (SD=0.0057)   
  pAN:proxy|AN:ALL (true=0)   |   rho=0.00053 (SD=0.0039)    |  r2_ps=0.00053 (SD=0.0039)   
 pAN:gene|AN:CHR1 (true=0.8)  |      rho=0.11 (SD=0.07)      |     r2_ps=0.11 (SD=0.07)     
pAN:proxy|AN:CHR1 (true=-0.2) |    rho=-0.041 (SD=0.012)     |   r2_ps=-0.041 (SD=0.012)    
   pAN:gene|AN:ODD (true=0)   |   rho=-0.0047 (SD=0.0079)    |  r2_ps=-0.0047 (SD=0.0079)   
  pAN:proxy|AN:ODD (true=0)   |   rho=0.00065 (SD=0.0081)    |  r2_ps=0.00065 (SD=0.0081)   
